## Proyecto final ML - Daniel Conrado Monroy Madrid 16012674

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
data = pd.read_csv('data_titanic_proyecto.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId           891 non-null int64
Name                  891 non-null object
Age                   714 non-null float64
SibSp                 891 non-null int64
Parch                 891 non-null int64
Ticket                891 non-null object
Fare                  891 non-null float64
Cabin                 204 non-null object
Embarked              889 non-null object
passenger_class       891 non-null object
passenger_sex         891 non-null object
passenger_survived    891 non-null object
dtypes: float64(2), int64(3), object(7)
memory usage: 83.6+ KB


Se aplica el criterio de utilizar features o características generales (se descarta PassengerId,SibSp,Parch,Ticket y Cabin (debido a la gran cantidad nan's que posee), por lo tanto, se decide utilizar las siguientes variables: 
1) Age,
2) Fare,
3) Embarked,
4) passenger_class,
5) passenger_sex y 
6) passenger_survived (que en realidad es la variable objetivo "y").

In [4]:
data.describe()

,PassengerId,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,29.699118,0.523008,0.381594,32.204208
std,257.353842,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,38.000000,1.000000,0.000000,31.000000
max,891.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
data['Embarked'].mode()

0    S
dtype: object

Como se puede observar en la info, Age tiene varios valores nan y Embarked cuenta con únicamente con 2. Para tratar el problema, se limpiará la variable Age llenando los espacios con la mediana y para la variable Embarked se  llenarán los espacios utilizando la moda (S = Southampton).

In [6]:
dataset = data.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis = 1)

In [7]:
cleanAge = dataset['Age'].fillna(dataset['Age'].median()) #Limpiar nan de Age, llenando con la mediana. 
dataset = dataset.drop(['Age'], axis=1)
dataset['Age'] = cleanAge

In [8]:
cleanEmbarked = dataset['Embarked'].astype(object).fillna('S') #Limpiar Embarked llenando con la moda. 
dataset = dataset.drop(['Embarked'], axis=1)
dataset['Embarked'] = cleanEmbarked

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Fare                  891 non-null float64
passenger_class       891 non-null object
passenger_sex         891 non-null object
passenger_survived    891 non-null object
Age                   891 non-null float64
Embarked              891 non-null object
dtypes: float64(2), object(4)
memory usage: 41.8+ KB


En este paso se asignan valores numéricos a las variables categóricas a través de Label Encoder:

In [10]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(dataset['Embarked'])
embarked = label_encoder.transform(dataset['Embarked'])
label_encoder.fit(dataset['passenger_sex'])
sex = label_encoder.transform(dataset['passenger_sex'])
label_encoder.fit(dataset['passenger_class'])
pclass = label_encoder.transform(dataset['passenger_class'])
label_encoder.fit(dataset['passenger_survived'])
psurvived = label_encoder.transform(dataset['passenger_survived'])

In [11]:
dataset = dataset.drop(['passenger_class','passenger_survived','Embarked', 'passenger_sex'], axis = 1)
dataset['Embarked'] = embarked
dataset['passenger_sex'] = sex
dataset['passenger_class'] = pclass
dataset['passenger_survived'] = psurvived

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Fare                  891 non-null float64
Age                   891 non-null float64
Embarked              891 non-null int32
passenger_sex         891 non-null int32
passenger_class       891 non-null int32
passenger_survived    891 non-null int32
dtypes: float64(2), int32(4)
memory usage: 27.9 KB


Ahora se normalizará la data:

In [12]:
from sklearn.preprocessing import normalize

In [13]:
features=dataset.iloc[:,0:2]

In [14]:
normalize=preprocessing.normalize(features)
normalize

array([[0.31298812, 0.94975704],
       [0.88244422, 0.47041706],
       [0.29156418, 0.95655127],
       ...,
       [0.64206767, 0.76664797],
       [0.75568908, 0.65493054],
       [0.2353827 , 0.97190276]])

In [15]:
Farenorm=normalize[:,0]
Agenorm=normalize[:,1]

In [16]:
dataset['Fare']=Farenorm
dataset['Age']=Agenorm

In [17]:
dataset.head()

,Fare,Age,Embarked,passenger_sex,passenger_class,passenger_survived
0,0.312988,0.949757,2,1,0,0
1,0.882444,0.470417,0,0,2,1
2,0.291564,0.956551,2,0,0,1
3,0.834942,0.550338,2,0,2,1
4,0.224148,0.974555,2,1,0,0


A continuación se separa la data en train, val y test:

In [135]:
y = dataset['passenger_survived'] 
y = pd.DataFrame(y,columns=['passenger_survived'])#Split data 
x = dataset.drop(['passenger_survived'],axis=1)

In [236]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [237]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [238]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(569, 5)
(143, 5)
(179, 5)
(569, 1)
(143, 1)
(179, 1)


Se importan las funciones que provee sklearn para el cálculo de métricas:

In [23]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

## 1. Decision tree

In [22]:
from sklearn import tree

In [24]:
def decisiontree(md,X,Y):
    clasificador_gini = tree.DecisionTreeClassifier(criterion = 'gini', max_depth=md)
    clasificador_gini.fit(X_train, y_train)
    yhat_gini = clasificador_gini.predict(X)
    accuracy = accuracy_score(Y,yhat_gini)
    error = 1 - accuracy
    reporte = classification_report(Y, yhat_gini)
    f1 = f1_score(Y,yhat_gini)
    precision = precision_score(Y,yhat_gini)
    recall = recall_score(Y,yhat_gini)
    metricas_array = np.array([accuracy,error,precision,recall,f1])
    df_metricas=pd.DataFrame(metricas_array,index=['Accuracy','Error','Precision','Recall','F1'])
    lista=[df_metricas,yhat_gini]
    return lista

In [27]:
dtreetrain1=decisiontree(3,X_train,y_train)
dtreetrain1[0]

,0
Accuracy,0.841828
Error,0.158172
Precision,0.904459
Recall,0.654378
F1,0.759358


In [28]:
dtreeval1=decisiontree(3,X_val,y_val)
dtreeval1[0]

,0
Accuracy,0.797203
Error,0.202797
Precision,0.828571
Recall,0.557692
F1,0.666667


In [29]:
dtreeval2=decisiontree(5,X_val,y_val)
dtreeval2[0]

,0
Accuracy,0.783217
Error,0.216783
Precision,0.769231
Recall,0.576923
F1,0.659341


In [35]:
dt = pd.concat([dtreetrain1[0],dtreeval1[0],dtreeval2[0]],axis=1).T
dt['Cadena de configuración'] = 'D Tree train,md=3','D Tree val,md=3','D Tree val,md=5'
dt['Modelo'] = 'Decision tree','Decision tree','Decision tree'
dt = dt[['Modelo','Cadena de configuración','Accuracy','Error','Precision','Recall','F1']]
dt

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,Decision tree,"D Tree train,md=3",0.841828,0.158172,0.904459,0.654378,0.759358
0,Decision tree,"D Tree val,md=3",0.797203,0.202797,0.828571,0.557692,0.666667
0,Decision tree,"D Tree val,md=5",0.783217,0.216783,0.769231,0.576923,0.659341


## 2. SVM

In [69]:
from sklearn.svm import SVC

In [75]:
def svm(X,Y,c):
    svm_titanic = SVC(kernel='linear',C=c)
    svm_titanic.fit(X_train, y_train['passenger_survived'])
    y_hat_svm = svm_titanic.predict(X)
    svm_accuracy = accuracy_score(Y, y_hat_svm)
    svm_error = 1 - svm_accuracy
    svm_precision =  precision_score(Y,y_hat_svm)
    svm_recall = recall_score(Y,y_hat_svm)
    svm_f1 = f1_score(Y,y_hat_svm)
    metricas_svm = np.array([svm_accuracy,svm_error,svm_precision,svm_recall,svm_f1])
    df_metricas=pd.DataFrame(metricas_svm,index=['Accuracy','Error','Precision','Recall','F1'])
    listasvm=[df_metricas,y_hat_svm]
    return listasvm

In [83]:
svmtrain1=svm(X_train,y_train,0.8)
svmtrain1[0]

,0
Accuracy,0.799649
Error,0.200351
Precision,0.758794
Recall,0.695853
F1,0.725962


In [80]:
svmval1=svm(X_val,y_val,0.8)
svmval1[0]

,0
Accuracy,0.748252
Error,0.251748
Precision,0.660000
Recall,0.634615
F1,0.647059


In [84]:
sv = pd.concat([svmtrain1[0],svmval1[0]],axis=1).T
sv['Cadena de configuración'] = 'SVM train1,C=0.8','SVM val1,C=0.8'
sv['Modelo'] = 'SVM','SVM'
sv = sv[['Modelo','Cadena de configuración','Accuracy','Error','Precision','Recall','F1']]

In [85]:
sv

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,SVM,"SVM train1,C=0.8",0.799649,0.200351,0.758794,0.695853,0.725962
0,SVM,"SVM val1,C=0.8",0.748252,0.251748,0.660000,0.634615,0.647059


## 3. Naive Bayes

Para poder calcular la tabla de frecuencias es preciso transformar las variables continuas en categóricas. Dicha tarea se logra -entre otros métodos- utilizando cuartiles para dividir a la data. La función de abajo realiza dicha tarea y devuelve un dataframe con las variables tranformadas.  

In [88]:
def dataq(data): ##Transformar variables continuas en categóricas usando cuartiles
    df=data
    datafare=pd.qcut(df['Fare'],4,labels=False)
    dataage=pd.qcut(df['Age'],4,labels=False)
    df=df.drop(['Fare','Age'],axis=1)
    df['Fare']=datafare
    df['Age']=dataage
    return df  #La funcion devuelve el dataframe transformado

In [89]:
xtrain_nb=dataq(X_train)
xval_nb=dataq(X_val)
xtest_nb=dataq(X_test)

In [90]:
%store xtrain_nb
%store xval_nb
%store xtest_nb

Stored 'xtrain_nb' (DataFrame)
Stored 'xval_nb' (DataFrame)
Stored 'xtest_nb' (DataFrame)


In [92]:
xtrain_nb.head() 

,Embarked,passenger_sex,passenger_class,Fare,Age
454,2,1,0,0,3
167,2,0,0,2,1
657,1,0,0,1,2
364,1,1,0,2,1
99,2,1,1,2,1


Ahora se construyen funciones para poder calcular las probabilidades y así construir la tabla de frecuencias. Dicho de otro modo, en esta parte es donde se "entrena" al modelo, utilizando la data de train para poder realizar los cálculos.

In [93]:
def embarkedp(x,y):
    ycount = np.bincount(y.iloc[:,0])
    no_count = ycount[0]
    yes_count = ycount[1]
    denominador_y = no_count + yes_count
    pno = no_count/denominador_y
    pyes = yes_count/denominador_y
    
    filtroe01 = (x['Embarked']==0)&(y['passenger_survived']==1)
    filtroe00 = (x['Embarked']==0)&(y['passenger_survived']==0)
    filtroe11 = (x['Embarked']==1)&(y['passenger_survived']==1)
    filtroe10 = (x['Embarked']==1)&(y['passenger_survived']==0)
    filtroe21 = (x['Embarked']==2)&(y['passenger_survived']==1)
    filtroe20 = (x['Embarked']==2)&(y['passenger_survived']==0)

    counte01 = np.bincount(filtroe01)[1]
    counte00 = np.bincount(filtroe00)[1]
    counte11 = np.bincount(filtroe11)[1]
    counte10 = np.bincount(filtroe10)[1]
    counte21 = np.bincount(filtroe21)[1]
    counte20 = np.bincount(filtroe20)[1]
    
    ## Probabilidades embarked (likelihood)
    pe01 = counte01/yes_count
    pe00 = counte00/no_count
    pe11 = counte11/yes_count
    pe10 = counte10/no_count
    pe21 = counte21/yes_count
    pe20 = counte20/no_count
    
    pey1 = np.array([pe01,pe11,pe21])
    pey0 = np.array([pe00,pe10,pe20])
    
    listape = [pey0,pey1]
    return listape   

In [94]:
pey0 = embarkedp(X_train,y_train)[0]
pey1 = embarkedp(X_train,y_train)[1]

In [95]:
%store pey0
%store pey1

Stored 'pey0' (ndarray)
Stored 'pey1' (ndarray)


In [96]:
def passengersexp(x,y):
    ycount = np.bincount(y.iloc[:,0])
    no_count = ycount[0]
    yes_count = ycount[1]
    denominador_y = no_count + yes_count
    pno = no_count/denominador_y
    pyes = yes_count/denominador_y
    
    filtrops01 = (x['passenger_sex']==0)&(y['passenger_survived']==1)
    filtrops00 = (x['passenger_sex']==0)&(y['passenger_survived']==0)
    filtrops11 = (x['passenger_sex']==1)&(y['passenger_survived']==1)
    filtrops10 = (x['passenger_sex']==1)&(y['passenger_survived']==0)

    countps01 = np.bincount(filtrops01)[1]
    countps00 = np.bincount(filtrops00)[1]
    countps11 = np.bincount(filtrops11)[1]
    countps10 = np.bincount(filtrops10)[1]
    
    ## Probabilidades passenger sex
    pps01 = countps01/yes_count
    pps00 = countps00/no_count
    pps11 = countps11/yes_count
    pps10 = countps10/no_count
    
    pps0 = np.array([pps00,pps10])
    pps1 = np.array([pps01,pps11])
    
    listaps = [pps0,pps1]
    return listaps

In [97]:
pps0 = passengersexp(X_train,y_train)[0]
pps1 = passengersexp(X_train,y_train)[1]

In [98]:
%store pps0
%store pps1

Stored 'pps0' (ndarray)
Stored 'pps1' (ndarray)


In [99]:
def passengerclassp(x,y):
    ycount = np.bincount(y.iloc[:,0])
    no_count = ycount[0]
    yes_count = ycount[1]
    denominador_y = no_count + yes_count
    pno = no_count/denominador_y
    pyes = yes_count/denominador_y
    
    filtropc01 = (x['passenger_class']==0)&(y['passenger_survived']==1)
    filtropc00 = (x['passenger_class']==0)&(y['passenger_survived']==0)
    filtropc11 = (x['passenger_class']==1)&(y['passenger_survived']==1)
    filtropc10 = (x['passenger_class']==1)&(y['passenger_survived']==0)
    filtropc21 = (x['passenger_class']==2)&(y['passenger_survived']==1)
    filtropc20 = (x['passenger_class']==2)&(y['passenger_survived']==0)

    countpc01 = np.bincount(filtropc01)[1]
    countpc00 = np.bincount(filtropc00)[1]
    countpc11 = np.bincount(filtropc11)[1]
    countpc10 = np.bincount(filtropc10)[1]
    countpc21 = np.bincount(filtropc21)[1]
    countpc20 = np.bincount(filtropc20)[1]
    
    ppc01 = countpc01/yes_count
    ppc00 = countpc00/no_count
    ppc11 = countpc11/yes_count
    ppc10 = countpc10/no_count
    ppc21 = countpc21/yes_count
    ppc20 = countpc20/no_count
    
    ppc0 = np.array([ppc00,ppc10,ppc20])
    ppc1 = np.array([ppc01,ppc11,ppc21])
    
    listapc = [ppc0,ppc1]
    return listapc

In [100]:
ppc0 = passengerclassp(X_train,y_train)[0]
ppc1 = passengerclassp(X_train,y_train)[1]

In [101]:
%store ppc0
%store ppc1

Stored 'ppc0' (ndarray)
Stored 'ppc1' (ndarray)


In [102]:
def farep(x,y):
    ycount = np.bincount(y.iloc[:,0])
    no_count = ycount[0]
    yes_count = ycount[1]
    denominador_y = no_count + yes_count
    pno = no_count/denominador_y
    pyes = yes_count/denominador_y
    
    filtrof01 = (x['Fare']==0)&(y['passenger_survived']==1)
    filtrof00 = (x['Fare']==0)&(y['passenger_survived']==0)
    filtrof11 = (x['Fare']==1)&(y['passenger_survived']==1)
    filtrof10 = (x['Fare']==1)&(y['passenger_survived']==0)
    filtrof21 = (x['Fare']==2)&(y['passenger_survived']==1)
    filtrof20 = (x['Fare']==2)&(y['passenger_survived']==0)
    filtrof31 = (x['Fare']==3)&(y['passenger_survived']==1)
    filtrof30 = (x['Fare']==3)&(y['passenger_survived']==0)

    countf01 = np.bincount(filtrof01)[1]
    countf00 = np.bincount(filtrof00)[1]
    countf11 = np.bincount(filtrof11)[1]
    countf10 = np.bincount(filtrof10)[1]
    countf21 = np.bincount(filtrof21)[1]
    countf20 = np.bincount(filtrof20)[1]
    countf31 = np.bincount(filtrof31)[1]
    countf30 = np.bincount(filtrof30)[1]

    #Probablilidades para Fare
    pf01 = countf01/yes_count
    pf00 = countf00/no_count
    pf11 = countf11/yes_count
    pf10 = countf10/no_count
    pf21 = countf21/yes_count
    pf20 = countf20/no_count
    pf31 = countf31/yes_count
    pf30 = countf30/no_count
    
    pf0 = np.array([pf00,pf10,pf20,pf30])
    pf1 = np.array([pf01,pf11,pf21,pf31])
    
    listapf = [pf0,pf1]
    return listapf   

In [103]:
pf0 = farep(xtrain_nb,y_train)[0]
pf1 = farep(xtrain_nb,y_train)[1]

In [106]:
%store pf0
%store pf1

Stored 'pf0' (ndarray)
Stored 'pf1' (ndarray)


In [104]:
def agep(x,y):
    ycount = np.bincount(y.iloc[:,0])
    no_count = ycount[0]
    yes_count = ycount[1]
    denominador_y = no_count + yes_count
    pno = no_count/denominador_y
    pyes = yes_count/denominador_y
    
    filtroa01 = (x['Age']==0)&(y['passenger_survived']==1)
    filtroa00 = (x['Age']==0)&(y['passenger_survived']==0)
    filtroa11 = (x['Age']==1)&(y['passenger_survived']==1)
    filtroa10 = (x['Age']==1)&(y['passenger_survived']==0)
    filtroa21 = (x['Age']==2)&(y['passenger_survived']==1)
    filtroa20 = (x['Age']==2)&(y['passenger_survived']==0)
    filtroa31 = (x['Age']==3)&(y['passenger_survived']==1)
    filtroa30 = (x['Age']==3)&(y['passenger_survived']==0)

    counta01 = np.bincount(filtroa01)[1]
    counta00 = np.bincount(filtroa00)[1]
    counta11 = np.bincount(filtroa11)[1]
    counta10 = np.bincount(filtroa10)[1]
    counta21 = np.bincount(filtroa21)[1]
    counta20 = np.bincount(filtroa20)[1]
    counta31 = np.bincount(filtroa31)[1]
    counta30 = np.bincount(filtroa30)[1]

    #Probablilidades para Age
    pa01 = counta01/yes_count
    pa00 = counta00/no_count
    pa11 = counta11/yes_count
    pa10 = counta10/no_count
    pa21 = counta21/yes_count
    pa20 = counta20/no_count
    pa31 = counta31/yes_count
    pa30 = counta30/no_count
    
    pa0 = np.array([pa00,pa10,pa20,pa30])
    pa1 = np.array([pa01,pa11,pa21,pa31])
    
    listapa = [pa0,pa1]
    return listapa   

In [107]:
pa0 = agep(xtrain_nb,y_train)[0]
pa1 = agep(xtrain_nb,y_train)[1]

In [108]:
%store pa0
%store pa1

Stored 'pa0' (ndarray)
Stored 'pa1' (ndarray)


In [109]:
def yp(y):
    ycount = np.bincount(y.iloc[:,0])
    no_count = ycount[0]
    yes_count = ycount[1]
    denominador_y = no_count + yes_count
    pno = no_count/denominador_y
    pyes = yes_count/denominador_y
    
    listapy = [pno,pyes]
    return listapy

In [123]:
pno_train=yp(y_train)[0]
pyes_train=yp(y_train)[1]

In [124]:
%store pno_train
%store pyes_train

Stored 'pno_train' (float64)
Stored 'pyes_train' (float64)


In [112]:
def replaceyes(x):    #la funcion crea un df con las probabilidades de cada x dado y=1; toma como parametro un dataset
    dict1={'Embarked':{0: pey1[0],1:pey1[1],2:pey1[2]}}
    dict2={'passenger_sex':{0: pps1[0],1:pps1[1]}}
    dict3={'passenger_class':{0: ppc1[0],1:ppc1[1],2:ppc1[2]}}
    dict4={'Fare':{0: pf1[0],1:pf1[1],2:pf1[2],3:pf1[3]}}
    dict5={'Age':{0: pa1[0],1:pa1[1],2:pa1[2],3:pa1[3]}}
    rep1=x.replace(dict1)['Embarked']
    rep2=x.replace(dict2)['passenger_sex']
    rep3=x.replace(dict3)['passenger_class']
    rep4=x.replace(dict4)['Fare']
    rep5=x.replace(dict5)['Age']
    lista=[rep1,rep2,rep3,rep4,rep5]
    df=pd.concat(lista,axis=1)
    return df

In [113]:
def replaceno(x):    #la funcion crea un df con las probabilidades de cada x dado y=0; toma como parametro un dataset
    dict1={'Embarked':{0: pey0[0],1:pey0[1],2:pey0[2]}}
    dict2={'passenger_sex':{0: pps0[0],1:pps0[1]}}
    dict3={'passenger_class':{0: ppc0[0],1:ppc0[1],2:ppc0[2]}}
    dict4={'Fare':{0: pf0[0],1:pf0[1],2:pf0[2],3:pf0[3]}}
    dict5={'Age':{0: pa0[0],1:pa0[1],2:pa0[2],3:pa0[3]}}
    rep1=x.replace(dict1)['Embarked']
    rep2=x.replace(dict2)['passenger_sex']
    rep3=x.replace(dict3)['passenger_class']
    rep4=x.replace(dict4)['Fare']
    rep5=x.replace(dict5)['Age']
    lista=[rep1,rep2,rep3,rep4,rep5]
    df=pd.concat(lista,axis=1)
    return df

In [292]:
def naivebayes(x,Y):
    numeradoryes=replaceyes(x)['Embarked']*replaceyes(x)['passenger_sex']*replaceyes(x)['passenger_class']*replaceyes(x)['Fare']*replaceyes(x)['Age']*pyes_train
    numeradorno=replaceno(x)['Embarked']*replaceno(x)['passenger_sex']*replaceno(x)['passenger_class']*replaceno(x)['Fare']*replaceno(x)['Age']*pno_train
    denominador=numeradoryes+numeradorno
    pyesx=numeradoryes/denominador
    pnox=numeradorno/denominador
    x['pyes']=pyesx
    x['pno']=pnox
    serieslist=[x['pno'],x['pyes']]
    df_probs=pd.concat(serieslist,axis=1)
    prob_hat=df_probs.max(axis=1)
    y_est=df_probs.idxmax(axis=1)
    yhat=y_est.replace({'pno': 0,'pyes': 1})
    accuracy = accuracy_score(Y,yhat)
    error = 1 - accuracy
    f1 = f1_score(Y,yhat)
    precision = precision_score(Y,yhat)
    recall = recall_score(Y,yhat)
    metricas_array = np.array([accuracy,error,precision,recall,f1])
    df_metricas=pd.DataFrame(metricas_array,index=['Accuracy','Error','Precision','Recall','F1'])
    lista=[df_metricas,yhat.values,prob_hat]
    return lista

In [128]:
nbtrain = naivebayes(xtrain_nb,y_train)
nbtrain[0]

,0
Accuracy,0.762742
Error,0.237258
Precision,0.688073
Recall,0.691244
F1,0.689655


In [129]:
nbval = naivebayes(xval_nb,y_val)
nbval[0]

,0
Accuracy,0.748252
Error,0.251748
Precision,0.642857
Recall,0.692308
F1,0.666667


In [130]:
nb = pd.concat([nbtrain[0],nbval[0]],axis=1).T
nb['Cadena de configuración'] = 'Naive Bayes train','Naive Bayes validation'
nb['Modelo'] = 'Naive Bayes','Naive Bayes'
nb = nb[['Modelo','Cadena de configuración','Accuracy','Error','Precision','Recall','F1']]

In [131]:
nb

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,Naive Bayes,Naive Bayes train,0.762742,0.237258,0.688073,0.691244,0.689655
0,Naive Bayes,Naive Bayes validation,0.748252,0.251748,0.642857,0.692308,0.666667


Referencia utilizada para realizar el modelo: https://www.geeksforgeeks.org/naive-bayes-classifiers/. La idea fue implementar el algoritmo paso a paso utilizando las funciones que ofrece Pandas para manipular dataframes. 

## 4. Regresión Logística

In [142]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(569, 5)
(143, 5)
(179, 5)
(569, 1)
(143, 1)
(179, 1)


In [143]:
def yrl(y): ##Esta se utiliza para convertir los pandas series a arrays para luego poder obtener las metricas
    ydata = y.values
    return ydata

In [147]:
ytrainrl=yrl(y_train)
yvalrl=yrl(y_val)
ytestrl=yrl(y_test)

In [148]:
%store ytrainrl
%store yvalrl
%store ytestrl

Stored 'ytrainrl' (ndarray)
Stored 'yvalrl' (ndarray)
Stored 'ytestrl' (ndarray)


A diferencia de los 3 modelos anteriores (2 implementados en sklearn y otro en Pandas), la regresión logística se trabajará en TensorFlow. La función del modelo recibe como parámetros la data x, y, el valor de learning rate (lr), el factor de regularizaión (fr), los epochs y el batch. 

In [146]:
def regresion_logistica(x,y,lr,fr,epoch,batch):
    session = tf.Session()
    X = tf.placeholder(shape=[None, 5], dtype = tf.float32)
    Y = tf.placeholder(shape = [None, 1], dtype=tf.float32)
    W = tf.Variable(tf.random_normal(shape=[5,1]))
    b = tf.Variable(tf.random_normal(shape=[1,1]))
    y_hat = tf.add(tf.matmul(X, W), b)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels = Y)) + fr*tf.nn.l2_loss(W)
    init = tf.global_variables_initializer()
    session.run(init)
    optimizador = tf.train.GradientDescentOptimizer(learning_rate=lr)
    opt_minimize = optimizador.minimize(cost)
    salida_prob = tf.sigmoid(y_hat)
    clasificacion_lr = tf.round(tf.sigmoid(y_hat))
    clasificacion_final = tf.cast(tf.equal(clasificacion_lr, Y), tf.float32)
    accuracy = tf.reduce_mean(clasificacion_final)
    epochs = epoch
    batch_size = batch
    for epoch in range(epochs):
        batches = int(X_train.shape[0] / batch_size)
        for i in range(batches):
            session.run(opt_minimize, feed_dict={X: X_train, Y: y_train})
            parametrosW = session.run(W,feed_dict={X: X_train, Y: y_train})
            parametrosb = session.run(b,feed_dict={X: X_train, Y: y_train}) #Hasta acá se entrena
            prob_output = session.run(salida_prob,feed_dict={X: x, Y: y})
            cc = session.run(clasificacion_final,feed_dict={X: x, Y: y})
            acc = session.run(accuracy, feed_dict={X: x, Y: y})
            errorlr = 1 - acc
            metricas_arraylr = np.array([acc,errorlr]) #accuracy en index 0; error en index 1.
            listlr=[metricas_arraylr,cc,prob_output,parametrosW,parametrosb]
            
    return listlr    

In [193]:
rltrainpred1 = regresion_logistica(X_train,ytrainrl,0.01,0.01,200,30)[1]
rltrainW = regresion_logistica(X_train,ytrainrl,0.01,0.01,200,30)[3]
rltrainb = regresion_logistica(X_train,ytrainrl,0.01,0.01,200,30)[4]
rltrainmetric = regresion_logistica(X_train,ytrainrl,0.01,0.01,200,30)[0]

In [195]:
rltrainpred2 = regresion_logistica(X_train,ytrainrl,0.001,10,200,30)[1]
rltrainW2 = regresion_logistica(X_train,ytrainrl,0.001,10,200,30)[3]
rltrainb2 = regresion_logistica(X_train,ytrainrl,0.001,10,200,30)[4]
rltrainmetric2 = regresion_logistica(X_train,ytrainrl,0.001,10,200,30)[0]

In [197]:
rltrainpred3 = regresion_logistica(X_train,ytrainrl,0.0001,5,200,30)[1]
rltrainW3 = regresion_logistica(X_train,ytrainrl,0.0001,5,200,30)[3]
rltrainb3 = regresion_logistica(X_train,ytrainrl,0.0001,5,200,30)[4]
rltrainmetric3 = regresion_logistica(X_train,ytrainrl,0.0001,5,200,30)[0]

In [200]:
rltrainpred4 = regresion_logistica(X_train,ytrainrl,0.0001,0.01,200,30)[1]
rltrainW4 = regresion_logistica(X_train,ytrainrl,0.0001,0.01,200,30)[3]
rltrainb4 = regresion_logistica(X_train,ytrainrl,0.0001,0.01,200,30)[4]
rltrainmetric4 = regresion_logistica(X_train,ytrainrl,0.0001,0.01,200,30)[0]

In [224]:
%store rltrainW4
%store rltrainb4

Stored 'rltrainW4' (ndarray)
Stored 'rltrainb4' (ndarray)


In [202]:
rlvalpred1 = regresion_logistica(X_val,yvalrl,0.0001,0.01,200,30)[1]

In [157]:
#funcion similar a los modelos anteriores: regresa df con metricas, yhats y la salida probabilistica (para este caso)
def rlfinal(y,yhat):
    prediccion = yhat
    accuracy = accuracy_score(y,prediccion.astype(int))  #las y que se ingresan son las obtenidas en la funcion
    error = 1 - accuracy
    precision = precision_score(y,prediccion.astype(int))
    recall = recall_score(y,prediccion.astype(int))
    f1 = f1_score(y,prediccion.astype(int))
    metricas_rl = np.array([accuracy,error,precision,recall, f1])
    df_metricasrl=pd.DataFrame(metricas_rl,index=['Accuracy','Error','Precision','Recall','F1'])
    listarl=[df_metricasrl,prediccion.astype(int).T]
    return listarl

In [194]:
rltrain1 = rlfinal(ytrainrl,rltrainpred1)
rltrain1[0]

,0
Accuracy,0.291740
Error,0.708260
Precision,0.293333
Recall,0.608295
F1,0.395802


In [196]:
rltrain2 = rlfinal(ytrainrl,rltrainpred2)
rltrain2[0]

,0
Accuracy,0.0
Error,1.0
Precision,0.0
Recall,0.0
F1,0.0


In [199]:
rltrain3 = rlfinal(ytrainrl,rltrainpred3)
rltrain3[0]

,0
Accuracy,0.0
Error,1.0
Precision,0.0
Recall,0.0
F1,0.0


In [201]:
rltrain4 = rlfinal(ytrainrl,rltrainpred4)
rltrain4[0]

,0
Accuracy,0.992970
Error,0.007030
Precision,0.995349
Recall,0.986175
F1,0.990741


In [203]:
rlval = rlfinal(yvalrl,rlvalpred1)
rlval[0]

,0
Accuracy,0.671329
Error,0.328671
Precision,0.542373
Recall,0.615385
F1,0.576577


In [206]:
rl = pd.concat([rltrain1[0],rltrain2[0],rltrain3[0],rltrain4[0],rlval[0]],axis=1).T
rl['Cadena de configuración'] = 'rl_train,lr=0.01,fr=0.01,epoch=200,batch=30','rl_train,lr=0.001,fr=10,epoch=200,batch=30','rl_train,lr=0.0001,fr=5,epoch=200,batch=30','rl_train,lr=0.0001,fr=0.01,epoch=200,batch=30','rl_val,lr=0.0001,fr=0.01,epoch=200,batch=30'
rl['Modelo'] = 'RL','RL','RL','RL','RL'
rl = rl[['Modelo','Cadena de configuración','Accuracy','Error','Precision','Recall','F1']]

In [207]:
rl

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,RL,"rl_train,lr=0.01,fr=0.01,epoch=200,batch=30",0.291740,0.708260,0.293333,0.608295,0.395802
0,RL,"rl_train,lr=0.001,fr=10,epoch=200,batch=30",0.000000,1.000000,0.000000,0.000000,0.000000
0,RL,"rl_train,lr=0.0001,fr=5,epoch=200,batch=30",0.000000,1.000000,0.000000,0.000000,0.000000
0,RL,"rl_train,lr=0.0001,fr=0.01,epoch=200,batch=30",0.992970,0.007030,0.995349,0.986175,0.990741
0,RL,"rl_val,lr=0.0001,fr=0.01,epoch=200,batch=30",0.671329,0.328671,0.542373,0.615385,0.576577


Ahora se genera la tabla con los experimentos. Dicha tabla será guardada en un archivo csv. 

In [216]:
tabla1=pd.concat([dt,sv],axis=0)
tabla2=pd.concat([tablaejemplo,nb],axis=0)
tabla_train_val=pd.concat([tabla2,rl],axis=0)
tabla_train_val.index = ['0','1','2','3','4','5','6','7','8','9','10','11']

In [217]:
tabla_train_val

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,Decision tree,"D Tree train,md=3",0.841828,0.158172,0.904459,0.654378,0.759358
1,Decision tree,"D Tree val,md=3",0.797203,0.202797,0.828571,0.557692,0.666667
2,Decision tree,"D Tree val,md=5",0.783217,0.216783,0.769231,0.576923,0.659341
3,SVM,"SVM train1,C=0.8",0.799649,0.200351,0.758794,0.695853,0.725962
4,SVM,"SVM val1,C=0.8",0.748252,0.251748,0.660000,0.634615,0.647059
5,Naive Bayes,Naive Bayes train,0.762742,0.237258,0.688073,0.691244,0.689655
6,Naive Bayes,Naive Bayes validation,0.748252,0.251748,0.642857,0.692308,0.666667
7,RL,"rl_train,lr=0.01,fr=0.01,epoch=200,batch=30",0.291740,0.708260,0.293333,0.608295,0.395802
8,RL,"rl_train,lr=0.001,fr=10,epoch=200,batch=30",0.000000,1.000000,0.000000,0.000000,0.000000
9,RL,"rl_train,lr=0.0001,fr=5,epoch=200,batch=30",0.000000,1.000000,0.000000,0.000000,0.000000


In [222]:
tabla_train_val.to_csv('experimentosML.csv')

## Comentarios sobre Bootstrap y k-folds

Bootstrap es una técnica de remuestreo con reemplazo que básicamente ayuda a incrementar el número de muestras de una población, lo que estadísticamente significa que la distribución de los datos, en la medida que el tamaño de la muestra aumenta, tiende a tener una distribución normal. En el proyecto pudimos haber generado más muestras tomando la data disponible para poder realizar inferencias, construir intervalos de confianza y estimar el sesgo y la varianza y analizar el trade-off entre ambas a medida que el modelo adquiría complejidad. 

Respecto a k-folds, en el proyecto hubiera sido de utilidad para evaluar al modelo enseñandole data desconocida de "folds" en los que se partió la data. El procedimiento hubiera sido el siguiente: la data se hubiera dividido en "k" grupos y luego, cada grupo por separado hubiera actuado como test set de los otros (por lo tanto, los grupos tambíen son training sets) y luego se hubiera entrenado al modelo para luego realizar predicciones con el test set. 

## Fase de predicción-prueba-evaluación final

Ahora se utilizará el test set para que el modelo se enfrente a data que no ha visto. Luego, se obtendrán todas las predicciones y se realizará una moda para una respuesta final (ensemble learning).


## 1. Decision tree

In [232]:
test_dt = decisiontree(3,X_test,y_test)[1]

## 2. SVM

In [233]:
test_svm = svm(X_test,y_test,0.8)[1]

## 3. Naive Bayes

In [239]:
test_nb = naivebayes(xtest_nb,y_test)[1]

## 4. Regresión logística

In [252]:
test_rl1 = regresion_logistica(X_test,y_test,0.0001,0.15,200,30)[1]

In [258]:
test_rl = rlfinal(ytestrl,test_rl1)[1].reshape(-1)

Ahora se juntan todos las predicciones en un dataframe para luego obtener la moda.

In [434]:
list_test = [pd.Series(test_dt),pd.Series(test_svm),pd.Series(test_nb.values),pd.Series(test_rl)]

In [435]:
df_test = pd.concat(list_test,axis=1,ignore_index=True)
df_test.columns = ['Decision Tree','SVM','Naive Bayes','Regresión Logística']

In [436]:
moda = df_test.mode(axis=1)

In [437]:
df_test = pd.concat([df_test,moda],axis=1)


In [439]:
df_test.columns = ['Decision Tree','SVM','Naive Bayes','Regresión Logística','Predicción final','moda2']

In [440]:
df_test

,Decision Tree,SVM,Naive Bayes,Regresión Logística,Predicción final,moda2
0,1,1,1,1,1.0,NaN
1,0,0,0,1,0.0,NaN
2,1,1,1,1,1.0,NaN
3,1,1,0,1,1.0,NaN
4,1,1,1,1,1.0,NaN
5,0,0,0,1,0.0,NaN
6,0,0,0,1,0.0,NaN
7,1,1,1,1,1.0,NaN
8,0,0,1,1,0.0,1.0
9,1,1,1,0,1.0,NaN


In [441]:
def metricas_ensemble(Y,yhat):
    accuracy = accuracy_score(Y,yhat)
    error = 1 - accuracy
    f1 = f1_score(Y,yhat)
    precision = precision_score(Y,yhat)
    recall = recall_score(Y,yhat)
    metricas_array = np.array([accuracy,error,precision,recall,f1])
    df_metricas=pd.DataFrame(metricas_array,index=['Accuracy','Error','Precision','Recall','F1'])
    return df_metricas

In [442]:
metricas_ensemble(y_test,df_test['Predicción final'])

,0
Accuracy,0.720670
Error,0.279330
Precision,0.755556
Recall,0.465753
F1,0.576271


## Conclusiones

El proyecto fue bastante retador porque nos obligó como estudiantes a combinar todo lo que aprendimos no solo en el curso de Statiscal learning I, sino trambién en el curso de Ciencia de Datos en Python. En lo personal, mi mayor dificultad fue implementar naive bayes y la regresión logística. El primero porque comprendía la parte matemática pero la codificación era una barrera para materializar el objetivo. El segundo porque tensorflow es una librería que me ha costado aprender a utilizar. Las lecciones que guardo son varias, sobre todo porque aprendí  reconocer mis deficiencias a la hora de implementar código. Quizás el modelo de naive bayes pudo haber sido más eficiente, entonces son áreas en las que debo trabajar. 

Un apunte importante es que durante el proyecto, y sobre todo en el modelo naive bayes me di cuenta de lo amplia que es Pandas y que muchas operaciones que yo pensaba en utilizar ciclos, Pandas las resolvía sin la complejidad detrás de construir el mismo. Sin duda es una herramienta imprescindible en el toolkit de un data scientist. 

En cuanto al accuracy de los modelos, la regresión logística fue la más dificl y la que peor desempeño tuvo. Hay mucho por aprender acerca de cómo escoger el nivel óptimo de los hiperparámetros. El SVM me hubiera gustado poder implementar todos los kernels, pero debido a limitaciones técnicas por mi computadora el proceso de carga era demasiado lento. 

Las conclusiones después de haber realizado estos modelos se basan en que sin duda, utilizar sklearn no presupone el esfuerzo y la complejidad propia de tensoflow. Sin embargo, es una herramienta atractiva con la cual se puede aprender sobre el funcionamiento de un algoritmo. De igual manera, en este trabajo se pudo reforzar la importancia de analizar la data antes de empezar a construir modelos. Seleccionar y transformar las variables fueron tareas interesantes, sobre todo la de convertir las variables continuas en variables categóricas (tarea en donde Pandas fue la herramienta más sencilla y rápida para hacerlo). 

La mayor recomendación es la de seguir aprendiendo a programar y a tratar de reforzar la lógica detrás de cada línea de código, lo cual exige que también se estudie y se entienda el fundamento matemático de lo que después se programa. 